In [1]:
import pandas as pd
import yfinance as yf

In [2]:
eth = yf.Ticker('ETH-USD')
eth_data = eth.history('10y')

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [3]:
eth_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-11-09 00:00:00+00:00,308.644989,329.451996,307.056000,320.884003,893249984,0.0,0.0
2017-11-10 00:00:00+00:00,320.670990,324.717987,294.541992,299.252991,885985984,0.0,0.0
2017-11-11 00:00:00+00:00,298.585999,319.453003,298.191986,314.681000,842300992,0.0,0.0
2017-11-12 00:00:00+00:00,314.690002,319.153015,298.513000,307.907990,1613479936,0.0,0.0
2017-11-13 00:00:00+00:00,307.024994,328.415009,307.024994,316.716003,1041889984,0.0,0.0
...,...,...,...,...,...,...,...
2024-02-06 00:00:00+00:00,2298.955078,2389.826172,2296.788574,2372.201904,9520885493,0.0,0.0
2024-02-07 00:00:00+00:00,2372.262695,2442.636230,2353.706055,2423.745117,9660628536,0.0,0.0
2024-02-08 00:00:00+00:00,2424.080078,2459.556641,2414.749512,2419.906494,9941841732,0.0,0.0


In [4]:
def SMA(values, n):
    return pd.Series(values).rolling(n).mean()

In [5]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [6]:
from backtesting import Backtest

bt = Backtest(eth_data, SmaCross, cash=10_000, commission=0.002)

In [7]:
stats = bt.run()

In [8]:
stats

Start                     2017-11-09 00:00...
End                       2024-02-10 00:00...
Duration                   2284 days 00:00:00
Exposure Time [%]                   97.680525
Equity Final [$]                119761.349068
Equity Peak [$]                 590088.285975
Return [%]                        1097.613491
Buy & Hold Return [%]              673.444712
Return (Ann.) [%]                   48.678278
Volatility (Ann.) [%]              143.544609
Sharpe Ratio                         0.339116
Sortino Ratio                        0.898863
Calmar Ratio                         0.606541
Max. Drawdown [%]                  -80.255558
Avg. Drawdown [%]                  -11.370197
Max. Drawdown Duration     1086 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                  120
Win Rate [%]                        40.833333
Best Trade [%]                      120.59089
Worst Trade [%]                     -34.77867
Avg. Trade [%]                    

In [9]:
bt.plot()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='1450', ...)